In [1]:
import livef1

In [2]:
session = livef1.get_session(
    season=2025,
    meeting_identifier="Pre-Season Testing",
    session_identifier="Practice 3"
)

00:47:14 - Selected meeting/session is:
	Meeting Offname : FORMULA 1 ARAMCO PRE-SEASON TESTING 2025
	Meeting Name : Pre-Season Testing
	Meeting Circuit Shortname : Sakhir
00:47:14 - Got the meeting.
00:47:14 - Selected meeting/session is:
	Session Name : Day 3
00:47:14 - The session was received successfully.


In [3]:
# session = livef1.get_session(
#     season=2024,
#     meeting_identifier="Monza",
#     session_identifier="Race"
# )

In [4]:
session.generate()

00:47:15 - Fetching data : 'SessionStatus'
00:47:15 - Fetching data : 'CarData.z'
00:47:15 - Fetching data : 'Position.z'
00:47:15 - 'SessionStatus' has been fetched and parsed
00:47:15 - Fetching data : 'TimingData'
00:47:16 - 'TimingData' has been fetched and parsed
00:47:19 - 'Position.z' has been fetched and parsed
00:47:21 - 'CarData.z' has been fetched and parsed
00:47:30 - Silver tables are being generated.


>> 1 {'lap_number': 35, 'lap_time': None, 'in_pit': Timedelta('0 days 04:20:37.141000'), 'pit_out': Timedelta('0 days 04:41:06.583000'), 'sector1_time': Timedelta('0 days 00:00:40.843000'), 'sector2_time': Timedelta('0 days 00:01:00.045000'), 'sector3_time': None, None: None, 'speed_I1': '200', 'speed_I2': '202', 'speed_FL': '283', 'speed_ST': '90', 'no_pits': 9, 'lap_start_time': Timedelta('0 days 04:09:02.289000')}
>> 0 {'lap_number': 68, 'lap_time': Timedelta('0 days 00:01:50.227000'), 'in_pit': Timedelta('0 days 04:59:06.032000'), 'pit_out': None, 'sector1_time': Timedelta('0 days 00:00:29.790000'), 'sector2_time': Timedelta('0 days 00:00:40.139000'), 'sector3_time': None, None: None, 'speed_I1': '239', 'speed_I2': '270', 'speed_FL': '280', 'speed_ST': '315', 'no_pits': 13, 'lap_start_time': Timedelta('0 days 04:41:54.963000')}
>> 0 {'lap_number': 70, 'lap_time': Timedelta('0 days 00:01:50.227000'), 'in_pit': None, 'pit_out': None, 'sector1_time': Timedelta('0 days 00:00:30.898000'

00:47:30 - 'laps' has been generated and saved to the silver lake. You can access it from 'session.laps'.


>> 0 {'lap_number': 49, 'lap_time': None, 'in_pit': None, 'pit_out': None, 'sector1_time': Timedelta('0 days 00:00:30.944000'), 'sector2_time': Timedelta('0 days 00:00:41.684000'), 'sector3_time': None, None: None, 'speed_I1': '234', 'speed_I2': '245', 'speed_FL': '274', 'speed_ST': '286', 'no_pits': 7, 'lap_start_time': Timedelta('0 days 08:44:10.118000')}
>> 0 {'lap_number': 66, 'lap_time': Timedelta('0 days 00:01:38.586000'), 'in_pit': None, 'pit_out': None, 'sector1_time': Timedelta('0 days 00:00:31.177000'), 'sector2_time': Timedelta('0 days 00:00:42.551000'), 'sector3_time': None, None: None, 'speed_I1': '232', 'speed_I2': '247', 'speed_FL': None, 'speed_ST': '298', 'no_pits': 11, 'lap_start_time': Timedelta('0 days 04:59:06.032000')}


KeyError: 'lap_start_date'

In [15]:
(session.laps["lap_start_time"] + session.first_datetime).fillna(session.session_start_datetime)

(session.laps["lap_start_time"] + session.first_datetime).isna()

ValueError: cannot reindex on an axis with duplicate labels